In [1]:
import qunatize_resnet20 as q_res
import quantize_layers as ql
q_res20_model = q_res.cifar_resnet20() 
train_options = {
    'optimizer': 'adam',
    'lr': 0.001,
    'epoch_num': 20,
    'from_logits':True,
}
solver = Solver(q_res20_model,data_set,train_options)
solver.train()

TypeError: __init__() missing 1 required positional argument: 'to_fix_fn'